In [ ]:
# Imports
import pandas as pd
import numpy as np
import random
import logging
import os
import json
import requests

from pathlib import Path
from datetime import datetime

In [ ]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
# Read configuration file
with open("data/config.json", "r") as f:
    config = json.load(f)
    
# Used to map carrier names to the ones BigSchedule uses and supports
with open("data/carrier_mapping.json", "r") as f:
    carrier_mapping = json.load(f)

# Bigschedule login
with open("data/bigschedules_login.json", "r") as f:
    bs_login = json.load(f)
    
# Prepare base information
# UNLOCODE to port name mapping
port_mapping = (
    pd.concat([pd.read_csv(p, usecols=[1, 2, 4, 5], engine='python', names=[
              'country', 'port', 'name', 'subdiv']) for p in Path('data').glob("*UNLOCODE CodeListPart*")])
    .query('port == port')
    .assign(
        uncode=lambda x: x.country.str.cat(x.port),
        full_name=lambda x: np.where(
            x.subdiv.notnull(), x.name.str.cat(x.subdiv, sep=", "), x.name)
    )
    .drop_duplicates('uncode')
    .set_index('uncode')
    .to_dict('index')
)

# Read the vessel delay tracking file
xl = pd.ExcelFile('Vessel Delay Tracking.xlsx')

### BSExtractor

In [ ]:
bigschedules_sheet = (
    xl.parse(pd.to_datetime(xl.sheet_names,
                            errors='coerce',
                            format='%d.%m.%Y').max().date().strftime('%d.%m.%Y'),
                            parse_dates=True)
                            .query(f"`Fwd Agent` in {[k for k,v in carrier_mapping.items() if v != '']}")
                            .replace({'Fwd Agent': carrier_mapping})
)

In [ ]:
# Get port name
bigschedules_sheet = bigschedules_sheet.assign(
    pol_name=lambda x: x['Port of Loading'].apply(
        lambda y: port_mapping.get(y)['name']),
    pod_name=lambda x: x['Port of discharge'].apply(
        lambda y: port_mapping.get(y)['name']),
)

In [ ]:
# Determine what searches need to be made (splitting of concerns amongst BigSchedules, MSC & G2)

In [ ]:
# Make the searches on the BigSchedules portal
'''
Takes in a list of dataframe of vessels, their carriers, POL, POD.
Shrinks the above dataframe to vessels & their carriers. Uses this new dataframe for querying BigSchedules.
Outputs an updated dataframe of vessels & their carriers with 2 additional columns updated_eta and updated_etd.
'''

### MSCExtractor

In [ ]:
# Get the MSC delay sheet
msc_delay_sheet = (xl.parse(pd.to_datetime(xl.sheet_names,
                            errors='coerce',
                            format='%d.%m.%Y').max().date().strftime('%d.%m.%Y'),
                            parse_dates=True)
                            .query(f"`Fwd Agent` in {['MSC']}")
                            .replace({'Fwd Agent': carrier_mapping})
)

In [ ]:
msc_delay_sheet

In [ ]:
msc_delay_sheet.drop_duplicates('Port of Loading')

In [ ]:
# Further filter by POL

In [ ]:
session = requests.Session()

Now I just need to figure out how to get the first cookie and use it in subsequent headers.

In [ ]:
def json_payload(port_of_loading):
    return {
        'mscCode': port_of_loading,
        'isCountry': False
    }

### MSC Arrival-Departure API

In [ ]:
url = "https://www.msc.com/Site/WebServices/RouteFinder.svc/PortActivity"
payload = {
    'mscCode': 'NZBLU',
    'isCountry': False
}
headers = {
    'Accept': '*/*',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
    'Content-Type': 'application/json; charset=UTF-8',
    'Origin': 'https://www.msc.com',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.msc.com/arrivals-departures',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-GB,en;q=0.9',
    'Cookie': 'CMSPreferredCulture=en-GB; ASP.NET_SessionId=tht5lkut0asln2goiskoagfe; UrlReferrer=https://www.google.com/; CurrentContact=8b0b2fea-705b-4a4f-b8bf-bb1cd6c982bc; CMSLandingPageLoaded=true; MSCAgencyId=115867; BIGipServerkentico.app~kentico_pool=439883018.20480.0000; _ga=GA1.2.1736073830.1597290148; _gid=GA1.2.1289141279.1597290148; _gcl_au=1.1.345060449.1597290148; __hstc=100935006.13bb76c8a78a8d0a203a993ffef3a3f6.1597290148282.1597290148282.1597290148282.1; __hssrc=1; hubspotutk=13bb76c8a78a8d0a203a993ffef3a3f6; _ym_uid=15972901491036911544; _ym_d=1597290149; _ym_isad=1; _ym_visorc_65601397=w; newsletter-signup-cookie=temp-hidden; _hjid=3e183004-f562-4048-8b60-daccdf9c187c; _hjUserAttributesHash=2c3b62a0e1cd48bdfd4d01b922060e19; _hjCachedUserAttributes={`"attributes`":{`"mscAgencyId`":`"115867`"},`"userId`":null}; OptanonAlertBoxClosed=2020-08-13T03:42:45.080Z; CMSCookieLevel=200; VisitorStatus=11062214903; CMSUserPage={`"TimeStamp`":`"2020-08-13T05:43:28.7403678+02:00`",`"LastPageDocumentID`":2969,`"LastPageNodeID`":6383,`"Identifier`":`"93e5bd8e-a77a-49db-b00e-1e3712608ca2`"}; TS0142aef9=0192b4b6225179b1baa3b4d270b71a4eee782a0192338173beabaa471f306c2a13fe854bf6a7ac08ac21924991864aa7728c54559023beabd273d82285d5f943202adb58da417d61813232e89b240828c090f890c6a74dc4adfec38513d13447be4b5b4404d69f964987b7917f731b858f0c9880a139994b98397c4aeb5bd60b0d0e38ec9e5f3c97b13fb184b4e068506e6086954f8a515f2b7239d2e5c1b9c70f61ca74f736355c58648a6036e9b5d06412389ac41221c5cb740df99c84dc2bfef4a530dbc5e2577c189212eebac723d9ee9f98030f4bc6ca7d824ab313ae5fdd1eaa9886; OptanonConsent=isIABGlobal=false&datestamp=Thu+Aug+13+2020+11%3A43%3A36+GMT%2B0800+(Singapore+Standard+Time)&version=5.9.0&landingPath=NotLandingPage&groups=1%3A1%2C2%3A1%2C3%3A1%2C4%3A1%2C0_53017%3A1%2C0_53020%3A1%2C0_53018%3A1%2C0_53019%3A1%2C101%3A1&AwaitingReconsent=false; _gat=1; _gat_local=1; __hssc=100935006.3.1597290148283; _gali=ui-id-2'
}

In [ ]:
response = session.request("POST", url, headers=headers, json=payload)
response.json()

In [ ]:
print(session.cookies.get_dict())

### MSC Search Schedules API

Need the PortOfLoadId and PortOfDischargeID for every port..

In [ ]:
url = "https://www.msc.com/api/schedules/search?WeeksOut=8&DirectRoutes=false&Date=2020-08-11&From=1034&To=221"
headers = {
  'Accept': 'application/json',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
  'Content-Type': 'application/json',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Dest': 'empty',
  'Referer': 'https://www.msc.com/search-schedules',
  'Accept-Language': 'en-GB,en;q=0.9',
  'Cookie': 'CMSPreferredCulture=en-GB; ASP.NET_SessionId=tht5lkut0asln2goiskoagfe; UrlReferrer=https://www.google.com/; CurrentContact=8b0b2fea-705b-4a4f-b8bf-bb1cd6c982bc; MSCAgencyId=115867; BIGipServerkentico.app~kentico_pool=439883018.20480.0000; _ga=GA1.2.1736073830.1597290148; _gid=GA1.2.1289141279.1597290148; _gcl_au=1.1.345060449.1597290148; __hstc=100935006.13bb76c8a78a8d0a203a993ffef3a3f6.1597290148282.1597290148282.1597290148282.1; hubspotutk=13bb76c8a78a8d0a203a993ffef3a3f6; __hssrc=1; _ym_uid=15972901491036911544; _ym_d=1597290149; _ym_isad=1; newsletter-signup-cookie=temp-hidden; _hjid=3e183004-f562-4048-8b60-daccdf9c187c; _hjUserAttributesHash=2c3b62a0e1cd48bdfd4d01b922060e19; _hjCachedUserAttributes={"attributes":{"mscAgencyId":"115867"},"userId":null}; OptanonAlertBoxClosed=2020-08-13T03:42:45.080Z; CMSCookieLevel=200; VisitorStatus=11062214903; TS0142aef9=0192b4b6225179b1baa3b4d270b71a4eee782a0192338173beabaa471f306c2a13fe854bf6a7ac08ac21924991864aa7728c54559023beabd273d82285d5f943202adb58da417d61813232e89b240828c090f890c6a74dc4adfec38513d13447be4b5b4404d69f964987b7917f731b858f0c9880a139994b98397c4aeb5bd60b0d0e38ec9e5f3c97b13fb184b4e068506e6086954f8a515f2b7239d2e5c1b9c70f61ca74f736355c58648a6036e9b5d06412389ac41221c5cb740df99c84dc2bfef4a530dbc5e2577c189212eebac723d9ee9f98030f4bc6ca7d824ab313ae5fdd1eaa9886; OptanonConsent=isIABGlobal=false&datestamp=Thu+Aug+13+2020+11%3A43%3A36+GMT%2B0800+(Singapore+Standard+Time)&version=5.9.0&landingPath=NotLandingPage&groups=1%3A1%2C2%3A1%2C3%3A1%2C4%3A1%2C0_53017%3A1%2C0_53020%3A1%2C0_53018%3A1%2C0_53019%3A1%2C101%3A1&AwaitingReconsent=false'
}

response = session.request("GET", url, headers=headers)
response.json()

In [ ]:
response.json()[0]['Sailings'][0]

In [ ]:
for i in range(len(response.json()[0]['Sailings'])):
    

### MSC countryID API

### G2Extractor

In [ ]:
class G2Extractor:
    def __init__(self, g2_file, xl, carrier_mapping):
        self.schedule = pd.read_excel(g2_file, skiprows=9, index_col='Unnamed: 0')
        self.delay_sheet = (xl.parse(pd.to_datetime(xl.sheet_names,
                                errors='coerce',
                                format='%d.%m.%Y').max().date().strftime('%d.%m.%Y'),
                                parse_dates=True)
                                .query(f"`Fwd Agent` in {['G2OCEAN']}")
                                .replace({'Fwd Agent': carrier_mapping}))
        self.g2_port_map = {
            'AUPTJ': 'Portland',
            'AUNTL': 'Newcastle',
            'AUGLT': 'Gladstone',
            'NZTWI': 'Bluff',
            'TWKHH': 'Kaohsiung',
            'KRINC': 'Inchon',
            'KRPUS': 'Busan',
            'JPYOK': 'Yokohama',
            'JPNGO': 'Nagoya',
            'JPOSA': 'Osaka',
            'JPTOY': 'Toyama',
            'JPIHA': 'Niihama',
            'HKHKG': 'Hong Kong',
            'CNSHA': 'Shanghai'
        }
        
    def get_updated_etd(self, row):
        try:
            # column_index_etd is the column number that points to the ETD
            column_index_etd = np.argwhere(self.schedule.columns.str.contains(row['Vessel']))[0][0] + 1
        except IndexError:
            return np.nan
        return self.schedule.loc[self.schedule.index == self.g2_port_map.get(row['Port of Loading'])].iloc[:, column_index_etd][0] 
    
    def get_updated_eta(self, row):
        try:
            # column_index_eta is the column number that points to the ETA
            column_index_eta = np.argwhere(self.schedule.columns.str.contains(row['Vessel']))[0][0]
        except IndexError:
            return np.nan
        return self.schedule.loc[self.schedule.index == self.g2_port_map.get(row['Port of discharge'])].iloc[:, column_index_eta][0]
    
    def extract(self):
        self.delay_sheet['updated_etd'] = self.delay_sheet.apply(self.get_updated_etd, axis=1)
        self.delay_sheet['updated_eta'] = self.delay_sheet.apply(self.get_updated_eta, axis=1)

### Final update

In [ ]:
class DelayReport:
    def __init__(self):
        # Read configuration file
        with open("data/config.json", "r") as f:
            self.config = json.load(f)
            
        # Used to map carrier names to the ones BigSchedules uses and supports
        with open("data/carrier_mapping.json", "r") as f:
            self.carrier_mapping = json.load(f)
        
        # BigSchedules login
        with open("data/bigschedules_login.json", "r") as f:
            self.bs_login = json.load(f)
        
        # Prepare base information
        # UNLOCODE to port name mapping
        self.port_mapping = (
            pd.concat([pd.read_csv(p, usecols=[1, 2, 4, 5], engine='python', names=[
                      'country', 'port', 'name', 'subdiv']) for p in Path('data').glob("*UNLOCODE CodeListPart*")])
            .query('port == port')
            .assign(uncode=lambda x: x.country.str.cat(x.port),
                    full_name=lambda x: np.where(x.subdiv.notnull(), x.name.str.cat(x.subdiv, sep=", "), x.name))
            .drop_duplicates('uncode')
            .set_index('uncode')
            .to_dict('index'))
        
        # Read the vessel delay tracking file
        self.xl = pd.ExcelFile('Vessel Delay Tracking.xlsx')
        # today_date = datetime.now().strftime('%d.%m.%Y')
        # if today_date not in self.xl.sheet_names:
        #     raise Exception(
        #         f"The script cannot find today's date ({today_date}) in the Vessel Delay Tracking.xlsx file provided. Please check that the sheets are correctly named - the script will only operate on a sheet with today's date.")
        
    def run_bs(self):
        if self.config.get('run_bs'):
            bs_extractor = BSExtractor()
            bs_extractor.extract()
        
    def run_msc(self):
        if self.config.get('run_msc'):
            self.msc_extractor = MSCExtractor(self.xl, self.carrier_mapping)
            self.msc_extractor.extract()
    
    def run_g2(self):
        if self.config.get('run_g2'):
            self.g2_extractor = G2Extractor(self.config.get('g2_filename'), self.xl, self.carrier_mapping)
            self.g2_extractor.extract()
    
    def assemble(self):
        # Assemble the final dataframe to update
        main_delay_sheet = self.xl.parse()

        # Add new columns to the right side of the dataframe
        new_columns = ['updated_etd', 'updated_eta', 'No. of days delayed ETD', 'No. of days delayed ETA', 'Reason of Delay']
        main_delay_sheet[new_columns] = pd.DataFrame([[pd.NaT for i in range(4)] + [np.nan]])

        if self.config.get('run_bs'):
            main_delay_sheet.update(self.bs_extractor.delay_sheet)
        
        if self.config.get('run_msc'):
            main_delay_sheet.update(self.msc_extractor.delay_sheet)
        
        if self.config.get('run_g2'):
            main_delay_sheet.update(self.g2_extractor.delay_sheet)

        # Calculate the deltas
        main_delay_sheet['No. of days delayed ETD'] = (main_delay_sheet.updated_etd
                                                       - pd.to_datetime(main_delay_sheet['ETD Date'])).dt.days
        main_delay_sheet['No. of days delayed ETA'] = (main_delay_sheet.updated_eta
                                                       - pd.to_datetime(main_delay_sheet['Disport ETA'])).dt.days

        # Format the dates correctly via strftime
        date_columns = ['ETD Date', 'Disport ETA', 'updated_etd', 'updated_eta']
        for column in date_columns:
            main_delay_sheet[column] = main_delay_sheet[column].dt.strftime('%d/%m/%Y')
        self.main_delay_sheet = main_delay_sheet.copy()
    
    def output(self):
        # Output the excel file
        saved_file = f"Vessel Delay Tracking - {date.today().strftime('%d.%m.%Y')}.xlsx"
        self.main_delay_sheet.to_excel(saved_file)
        # os.startfile(saved_file)

In [ ]:
# Delay report skeleton
delay_report = DelayReport()
delay_report.run_bs()
delay_report.run_msc()
delay_report.run_g2()
delay_report.assemble()